In this notebook we explore the possibilities of fuzzy search algorithms in finding similarities.

**Classification using fuzzy matching**

-Classify whether question pairs are duplicate or not

-Let us start with importing the necessary modules for exploring the data.

In [67]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from pyxdameraulevenshtein import damerau_levenshtein_distance, normalized_damerau_levenshtein_distance
from nltk.stem.porter import *
stemmer = PorterStemmer()
import random
import re
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams
eng_stopwords = set(stopwords.words('english'))

random.seed(1337)

df_train = pd.read_csv('../data/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('../data/test.csv', encoding="ISO-8859-1")

num_train = df_train.shape[0]
print (num_train)

404290


In [68]:
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [69]:
df_test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


We use Jaccard similarity and Levenshtein distance to find the similarity between the questions. The number of words matched are also used as features for prediction.

In [70]:

df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

def get_unigrams(que):
    return [word for word in word_tokenize(que.lower()) if word not in eng_stopwords]

def get_common_unigrams(row):
    return len( set(row["unigrams_ques1"]).intersection(set(row["unigrams_ques2"])) )

def get_common_unigram_ratio(row):
    return float(row["unigrams_common_count"]) / max(len( set(row["unigrams_ques1"]).union(set(row["unigrams_ques2"])) ),1)

df_all["unigrams_ques1"] = df_all['question1'].apply(lambda x: get_unigrams(str(x)))
df_all["unigrams_ques2"] = df_all['question2'].apply(lambda x: get_unigrams(str(x)))
df_all["unigrams_common_count"] = df_all.apply(lambda row: get_common_unigrams(row),axis=1)
df_all["unigrams_common_ratio"] = df_all.apply(lambda row: get_common_unigram_ratio(row), axis=1)
print ("Unigrams generated...")

def get_bigrams(que):
    return [i for i in ngrams(que, 2)]

def get_common_bigrams(row):
    return len( set(row["bigrams_ques1"]).intersection(set(row["bigrams_ques2"])) )

def get_common_bigram_ratio(row):
    return float(row["bigrams_common_count"]) / max(len( set(row["bigrams_ques1"]).union(set(row["bigrams_ques2"])) ),1)

df_all["bigrams_ques1"] = df_all["unigrams_ques1"].apply(lambda x: get_bigrams(x))
df_all["bigrams_ques2"] = df_all["unigrams_ques2"].apply(lambda x: get_bigrams(x)) 
df_all["bigrams_common_count"] = df_all.apply(lambda row: get_common_bigrams(row),axis=1)
df_all["bigrams_common_ratio"] = df_all.apply(lambda row: get_common_bigram_ratio(row), axis=1)

df_all["words_ques1"] = df_all["unigrams_ques1"].apply(len)
df_all["words_ques2"] = df_all["unigrams_ques2"].apply(len)
print ("Bigrams generated...")
from nltk.corpus import stopwords

stops = set(stopwords.words("english"))

def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

print ("Word match share over...")

Unigrams generated...
Bigrams generated...
Word match share over...


In [71]:
train_qs = pd.Series(df_train['question1'].tolist() + df_train['question2'].tolist()).astype(str)
test_qs = pd.Series(df_test['question1'].tolist() + df_test['question2'].tolist()).astype(str)
dist_train = train_qs.apply(lambda x: len(x.split(' ')))
dist_test = test_qs.apply(lambda x: len(x.split(' ')))

from collections import Counter

# If a word appears only once, we ignore it completely (likely a typo)
# Epsilon defines a smoothing constant, which makes the effect of extremely rare words smaller
def get_weight(count, eps=10000, min_count=2):
    if count < min_count:
        return 0
    else:
        return 1 / (count + eps)

eps = 5000 
words = (" ".join(train_qs)).lower().split()
counts = Counter(words)
weights = {word: get_weight(count) for word, count in counts.items()}

def tfidf_word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    
    shared_weights = [weights.get(w, 0) for w in q1words.keys() if w in q2words] + [weights.get(w, 0) for w in q2words.keys() if w in q1words]
    total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]
    
    R = np.sum(shared_weights) / np.sum(total_weights)
    return R

In [72]:
def str_stem(str1):
    str1 = str(str1)
    str1 = re.sub(r'[^a-zA-Z0-9 ]',r'',str1)
    str1 = str1.lower()
    #str1 = (" ").join([stemmer.stem(z) for z in str1.split(" ")])
    return str1

def str_common_word(str1, str2):
    str1, str2 = str1.lower(), str2.lower()
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt
def ngram(tokens, n):
    grams =[tokens[i:i+n] for i in range(len(tokens)-(n-1))]
    return grams

def get_sim(a_tri,b_tri):
    intersect = len(set(a_tri) & set(b_tri))
    union = len(set(a_tri) | set(b_tri))
    if union == 0:
        return 0
    return float(intersect)/(union)

def jaccard_similarity(str1,str2):
    sentence_gram1 = str1
    sentence_gram2 = str2
    grams1 = ngram(sentence_gram1, 5)
    grams2 = ngram(sentence_gram2, 5)
    similarity = get_sim(grams1, grams2)
    return similarity
    

df_all['question1'] = df_all['question1'].map(lambda x:str_stem(x))
df_all['question2'] = df_all['question2'].map(lambda x:str_stem(x))

df_all['len_of_q1'] = df_all['question1'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_q2'] = df_all['question2'].map(lambda x:len(x.split())).astype(np.int64)

df_all['questions'] = df_all['question1']+"|"+df_all['question2']
print ("Questions combined...")
df_all['q2_in_q1'] = df_all['questions'].map(lambda x:str_common_word(x.split('|')[0],x.split('|')[1]))
df_all['q1_in_q2'] = df_all['questions'].map(lambda x:str_common_word(x.split('|')[1],x.split('|')[0]))
print ("Common words found ...")
df_all['jaccard'] = df_all['questions'].map(lambda x:jaccard_similarity(x.split('|')[0],x.split('|')[1]))
print ("Jaccard similarities computed...")
df_all['lev_distance'] = df_all['questions'].map(lambda x:normalized_damerau_levenshtein_distance(x.split('|')[0],x.split('|')[1]))
print ("Levenshtein distances computed...")

Questions combined...
Common words found ...
Jaccard similarities computed...
Levenshtein distances computed...


In [74]:
df_all.to_csv("../data/train_test.csv")
df_all.head()

,id,is_duplicate,qid1,qid2,question1,question2,test_id,unigrams_ques1,unigrams_ques2,unigrams_common_count,...,bigrams_common_ratio,words_ques1,words_ques2,len_of_q1,len_of_q2,questions,q2_in_q1,q1_in_q2,jaccard,lev_distance
0,0.0,0.0,1.0,2.0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,NaN,"[step, step, guide, invest, share, market, ind...","[step, step, guide, invest, share, market, ?]",6,...,0.625000,8,7,14,12,what is the step by step guide to invest in sh...,13,12,0.862069,0.138462
1,1.0,0.0,3.0,4.0,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,NaN,"[story, kohinoor, (, koh-i-noor, ), diamond, ?]","[would, happen, indian, government, stole, koh...",6,...,0.307692,7,12,8,13,what is the story of kohinoor kohinoor diamond...,5,6,0.200000,0.506024
2,2.0,0.0,5.0,6.0,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,NaN,"[increase, speed, internet, connection, using,...","[internet, speed, increased, hacking, dns, ?]",3,...,0.000000,7,6,14,10,how can i increase the speed of my internet co...,7,4,0.184466,0.555556
3,3.0,0.0,7.0,8.0,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,NaN,"[mentally, lonely, ?, solve, ?]","[find, remainder, [, math, ], 23^, {, 24, }, [...",1,...,0.000000,5,15,11,9,why am i mentally very lonely how can i solve ...,2,0,0.000000,0.800000
4,4.0,0.0,9.0,10.0,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,NaN,"[one, dissolve, water, quikly, sugar, ,, salt,...","[fish, would, survive, salt, water, ?]",3,...,0.000000,13,6,13,7,which one dissolve in water quikly sugar salt ...,4,4,0.084211,0.698630


In [81]:
#df_all = df_all.drop(['id','qid1','qid2','questions','unigrams_ques1','unigrams_ques2','bigrams_ques1','bigrams_ques2'],axis=1)
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]

df_train['word_match'] = df_train.apply(word_match_share, axis=1, raw=True)
df_train['tfidf_word_match'] = df_train.apply(tfidf_word_match_share, axis=1, raw=True)
df_test['word_match'] = df_test.apply(word_match_share, axis=1, raw=True)
df_test['tfidf_word_match'] = df_test.apply(tfidf_word_match_share, axis=1, raw=True)
id_test = df_test['test_id']


y_train = df_train['is_duplicate'].values
X_train = df_train.drop(['test_id','is_duplicate','question1','question2'],axis=1).values
X_test = df_test.drop(['test_id','is_duplicate','question1','question2'],axis=1).values


/home/srsasank/.local/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/srsasank/.local/lib/python3.5/site-packages/ipykernel/__main__.py:37: RuntimeWarning: invalid value encountered in double_scalars
/home/srsasank/.local/lib/python3.5/site-packages/ipykernel/__main__.py:37: RuntimeWarning: invalid value encountered in long_scalars
/home/srsasank/.local/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/srsasank/.local/lib/python3.5/si

In [82]:
df_train.head()

,is_duplicate,question1,question2,test_id,unigrams_common_count,unigrams_common_ratio,bigrams_common_count,bigrams_common_ratio,words_ques1,words_ques2,len_of_q1,len_of_q2,q2_in_q1,q1_in_q2,jaccard,lev_distance,word_match,tfidf_word_match
0,0.0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,NaN,6,0.857143,5,0.625000,8,7,14,12,13,12,0.862069,0.138462,0.909091,0.950123
1,0.0,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,NaN,6,0.461538,4,0.307692,7,12,8,13,5,6,0.200000,0.506024,0.363636,0.448403
2,0.0,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,NaN,3,0.300000,0,0.000000,7,6,14,10,7,4,0.184466,0.555556,0.363636,0.355658
3,0.0,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,NaN,1,0.062500,0,0.000000,5,15,11,9,2,0,0.000000,0.800000,0.000000,0.000000
4,0.0,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,NaN,3,0.200000,0,0.000000,13,6,13,7,4,4,0.084211,0.698630,0.266667,0.269607


In [86]:

from sklearn.cross_validation import train_test_split

x_trainb, x_validb, y_trainb, y_validb = train_test_split(X_train, y_train, test_size=0.2, random_state=4747)

import xgboost as xgb

params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.1
params['max_depth'] = 6

d_train = xgb.DMatrix(x_trainb, label=y_trainb)
d_valid = xgb.DMatrix(x_validb, label=y_validb)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=50, verbose_eval=10)

[0]	train-logloss:0.662423	valid-logloss:0.662333
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.524766	valid-logloss:0.524528
[20]	train-logloss:0.486221	valid-logloss:0.486441
[30]	train-logloss:0.472478	valid-logloss:0.473256
[40]	train-logloss:0.465876	valid-logloss:0.467035
[50]	train-logloss:0.462341	valid-logloss:0.464068
[60]	train-logloss:0.459554	valid-logloss:0.461633
[70]	train-logloss:0.456873	valid-logloss:0.459554
[80]	train-logloss:0.454488	valid-logloss:0.457767
[90]	train-logloss:0.452343	valid-logloss:0.456232
[100]	train-logloss:0.45051	valid-logloss:0.45503
[110]	train-logloss:0.448934	valid-logloss:0.454021
[120]	train-logloss:0.447204	valid-logloss:0.452908
[130]	train-logloss:0.445677	valid-logloss:0.451965
[140]	train-logloss:0.444519	valid-logloss:0.451317
[150]	train-logloss:0.443428	valid-logloss:0.450746
[160]	train-logloss:0.442263	v

In [87]:
d_test = xgb.DMatrix(X_test)
p_test = bst.predict(d_test)

sub = pd.DataFrame()
sub['test_id'] = np.int32(id_test)
sub['is_duplicate'] = p_test
sub.to_csv('../submission/simple_xgb_v2.csv', index=False)

This is my first notebook in Kaggle. Looking for suggestions to improve the model.

In [89]:
X_train2 = df_train.drop(['test_id','is_duplicate','question1','question2'],axis=1)
pos_train = X_train2[y_train == 1]
neg_train = X_train2[y_train == 0]

# Now we oversample the negative class
# There is likely a much more elegant way to do this...
p = 0.165
scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    neg_train = pd.concat([neg_train, neg_train])
    scale -=1
neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / (len(pos_train) + len(neg_train)))

x_train = pd.concat([pos_train, neg_train])
y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()
del pos_train, neg_train

0.19124366100096607


In [90]:
# Finally, we split some of the data off for validation
from sklearn.cross_validation import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)

In [102]:
import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.2
params['max_depth'] = 4
params['gamma'] =5
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 5000, watchlist, early_stopping_rounds=50, verbose_eval=10)

[0]	train-logloss:0.605861	valid-logloss:0.606339
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.395065	valid-logloss:0.39737
[20]	train-logloss:0.374835	valid-logloss:0.37761
[30]	train-logloss:0.369183	valid-logloss:0.372378
[40]	train-logloss:0.364991	valid-logloss:0.368408
[50]	train-logloss:0.362436	valid-logloss:0.366212
[60]	train-logloss:0.360412	valid-logloss:0.364427
[70]	train-logloss:0.358777	valid-logloss:0.362943
[80]	train-logloss:0.357776	valid-logloss:0.362138
[90]	train-logloss:0.356655	valid-logloss:0.361191
[100]	train-logloss:0.355655	valid-logloss:0.360372
[110]	train-logloss:0.354762	valid-logloss:0.359672
[120]	train-logloss:0.35407	valid-logloss:0.359099
[130]	train-logloss:0.353003	valid-logloss:0.358293
[140]	train-logloss:0.352487	valid-logloss:0.357926
[150]	train-logloss:0.351797	valid-logloss:0.357441
[160]	train-logloss:0.35125	val

In [101]:
X_test = df_test.drop(['test_id','is_duplicate','question1','question2'],axis=1)
d_test = xgb.DMatrix(X_test)
p_test = bst.predict(d_test)

sub = pd.DataFrame()
sub['test_id'] = np.int32(id_test)
sub['is_duplicate'] = p_test
sub.to_csv('../submission/simple_xgb_v5.csv', index=False)